In [ ]:
import opendatasets as od

In [ ]:
datadase_url = "https://www.kaggle.com/datasets/inputblackboxoutput/passwordevaluation"
od.download(datadase_url)

In [1]:
import pandas as pd
from joblib import dump

In [2]:
dataset_path = "./passwordevaluation/dataset.csv"
raw_df = pd.read_csv(dataset_path, encoding="utf-8")
raw_df.head()

,0,123456
0,0,12345
1,0,123456789
2,0,password
3,0,iloveyou
4,0,princess


In [3]:
raw_df.columns = ["strength", "password"]
raw_df = raw_df.astype({"strength": int, "password": str})
raw_df.dropna(inplace=True)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14344389 entries, 0 to 14344388
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   strength  int32 
 1   password  object
dtypes: int32(1), object(1)
memory usage: 164.2+ MB


In [4]:
df1 = raw_df[~raw_df["password"].str.contains(r"[^\x00-\x7F]")]

In [5]:
n_ = df1["strength"].value_counts().min()
df2 = df1.groupby("strength").sample(n=n_, random_state=42)
N = n_ * len(df2["strength"].unique())

In [6]:
df = df2.sample(N,random_state=24)
df.reset_index(drop=True, inplace=True)

In [8]:
df

,strength,password
0,0,using
1,2,stacey13041996
2,1,mccaffrey87
3,3,mitaya2003
4,4,vayacontioz
...,...,...
149360,1,alexisgt
149361,4,tai007nakab
149362,4,matricula1962
149363,0,p3ters
